<a href="https://colab.research.google.com/github/Gau-6426/PCAP/blob/main/Code/segment_status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_segments_status = pd.read_csv("https://raw.githubusercontent.com/Gau-6426/PCAP/main/Data/segment_status.csv")
print(df_segments_status.head(10))
df_segments_status.shape

   _id                updated_at  segment_id  velocity
0    0  2020-07-03T14:55:31.869Z       24845        20
1    1  2020-07-03T15:02:56.048Z       33923        10
2    2  2020-07-04T08:15:52.696Z       33824         5
3    3  2020-07-04T08:15:59.903Z       33824         5
4    4  2020-07-04T08:16:08.201Z       33824         5
5    5  2020-07-04T08:16:11.070Z       33824         5
6    6  2020-07-04T08:16:18.512Z       33824         5
7    7  2020-07-04T08:16:21.798Z       33824         5
8    8  2020-07-04T08:16:24.746Z       33824         5
9    9  2020-07-04T08:16:27.606Z       33824         5


(90938, 4)

In [ ]:
segment_status_sort=df_segments_status.sort_values(by='velocity')
segment_status_sort

,_id,updated_at,segment_id,velocity
32416,32416,2020-08-03T00:20:31.797Z,29164,0
12828,12828,2020-08-03T00:05:30.888Z,83584,0
8119,8119,2020-08-03T00:02:28.071Z,6536,0
7466,7466,2020-08-02T23:59:32.926Z,62128,0
33760,33760,2020-08-03T00:23:27.901Z,5562,0
...,...,...,...,...
50856,50856,2020-12-02T15:33:41.160Z,14409,199
42002,42002,2020-11-26T01:00:22.535Z,63453,199
52438,52438,2020-12-04T02:19:19.741Z,68714,200
49320,49320,2020-12-02T02:21:46.673Z,10099,200
